# Исследование надежности заемщиков

# Цель исследования

Определить, как разные характеристики клиента влияют на факт погашения кредита в срок.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

- Необходимо ответить на следующие вопросы:
-- Есть ли зависимость между количеством детей и возвратом кредита в срок?
-- Есть ли зависимость между семейным положением и возвратом кредита в срок?
-- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
-- Как разные цели кредита влияют на его возврат в срок?

## Загрузка библиотек и обзор данных

### Загрузка библиотек

In [27]:
import pandas as pd

### Считаем данные из csv-файла в датафрейм и сохраним в переменную data

In [28]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

### Выводим первые 5 строчек датафрейма data на экран

In [29]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Выводим основную информацию о датафрейме с помощью метода info

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов. Тип данных во всех столбцах — object, float и int.

Согласно документации к данным:

- children — количество детей в семье;
- days_employed — общий трудовой стаж в днях;
- dob_years — возраст клиента в годах;
- education — уровень образования клиента;
- education_id — идентификатор уровня образования;
- family_status — семейное положение;
- family_status_id — идентификатор семейного положения;
- gender — пол клиента;
- income_type — тип занятости;
- debt — имел ли задолженность по возврату кредитов;
- total_income — ежемесячный доход;
- purpose — цель получения кредита;

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

In [ ]:
df.describe()

В каждой строке таблицы — данные о заёмщике. Предварительно можно утверждать, что информации достаточно для ответов на вопросы.

Встречаются пропуски в данных, заметны артефакты (например, отрицательное количество дней стажа), возможны дубликаты, и, скорее всего, потребуется изменение формата в колонках для удобства.

## Предобработка данных

### Выводим количество пропущенных значений для каждого столбца

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Cтолбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. 

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработаем значения в столбце трудового стажа: заменим все отрицательные значения положительными с помощью метода abs

In [7]:
data['days_employed'] = data['days_employed'].abs()

### Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) аномально большие значения

### Выведем перечень уникальных значений столбца children

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

### Удаление строк, в которых встречаются аномальные значения в столбце с детьми

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

### Убедимся, что артефакты удалены

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Заполним пропуски в столбце days_employed медианными значениями каждого типа занятости income_type

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

### Убедимся, что все пропуски заполнены

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замените вещественный тип данных в столбце total_income на целочисленный

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Приведем к нижнему регистру столбец education

In [15]:
data['education'] = data['education'].str.lower()

### Выведите на экран количество строк-дубликатов в данных

In [16]:
data.duplicated().sum()

71

### Удаление дубликатов

In [17]:
data = data.drop_duplicates()

## Категоризация данных

### На основании диапазонов, указанных ниже, создаем в датафрейме столбец total_income_category с категориями:
- 0–30000 — 'E'
- 30001–50000 — 'D'
- 50001–200000 — 'C'
- 200001–1000000 — 'B'
- 1000001 и выше — 'A'

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

### Выведем перечень уникальных целей взятия кредита

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- 'операции с автомобилем'
- 'операции с недвижимостью'
- 'проведение свадьбы'
- 'получение образования'

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

### Проверка на зависимость между количеством детей и возвратом кредита в срок

In [23]:
data_new = data.pivot_table(index = "children" , values = "debt" , aggfunc = ["sum" , "count" , "mean"])
data_new

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


- 75% без детей
- 92% с одним ребенком
- 81% с двумя детьми
- 81% c тремя детьми
- 97% с четырьмя детьми
- 100% с 5 детьми

Так как выборки несбалансированны, мы с уверенностью можем сравнить самые большие (сопоставимые по размеру категории заемщиков). Сравнивая категрии с без детей, с одним ребенком или двумя, можно сделать вывод, что с увеличением количества детей, растет процент заемщиков без долга.

### Проверка на зависимость между семейным положением и возвратом кредита в срок

In [24]:
data_new = data.pivot_table(index = "family_status" , values = "debt" , aggfunc = ["sum" , "count" , "mean"])
data_new

,sum,count,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,273,2796,0.097639
в разводе,84,1189,0.070648
вдовец / вдова,63,951,0.066246
гражданский брак,385,4134,0.093130
женат / замужем,927,12261,0.075606


- 97% для не женатых
- 93% для гражданских браков
- 75% для женатых и замужних
- 70% в разводе
- 66% для вдов и вдавцов

Можно утверждать, что заемщики состоящие или состоявшие в браке, более надежны.

### Проверка на зависимость между уровнем дохода и возвратом кредита в срок

In [25]:
data_new = data.pivot_table(index = "total_income_category" , values = "debt" , aggfunc = ["sum" , "count" , "mean"])
data_new

,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,354,5014,0.070602
C,1353,15921,0.084982
D,21,349,0.060172
E,2,22,0.090909


Если сравнить только две самые большие категории C (50 - 200) и B (200 - 1 млн), выходит что заемщики с более высоким доходом являются более надежными.

### Определим как разные цели кредита влияют на его возврат в срок

In [26]:
data_new = data.pivot_table(index = "purpose_category", values = "debt" , aggfunc = ["sum" , "count" , "mean"])
data_new

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,4279,0.093480
операции с недвижимостью,780,10751,0.072551
получение образования,369,3988,0.092528
проведение свадьбы,183,2313,0.079118


Самые рискованные кредиты идут на недвижимость и свадьбы.
Кредиты на автомобили и образование люди стараются вернуть в срок.

### Определим возможные причины появления пропусков в исходных данных

Пропуски встрачаются в двух столбцах - дни трудового стажа и доходы.

Дни трудового стажа: человек может не работать, уволиться, скорее всего этот столбец считается автоматически.

В столбце с доходом человек также может уволиться и в настоящее время не работать.

Самый вероятные причины - человеческий фактор или техническая ошибка.

### Определим, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных

Возможно сработает и среднее значение, если нет анамольно больших и маленьких значений. 

В случае, когда в данных большой разброс значений - медиана покажет более корректный результат и усреднит значения, так как медиана менее чувствительна к выбросам.

## Вывод

За основу проекта был взят датасет статистики платежестпособности клиентов банка.

Цель исследования: Поиск закономерностей между семейным положением и колличеством детей при погашении кредита. 

При первичном ознакомлении с данными были выявлены следующие недостатки - достаточно большое колличество пропусков (2174 пропусков в стлобцах дохода и рабочего стажа). Методом поиска медианы были добавлены недостающие значения в столбец доходов и трудового стажа (для трудового стажа использовалась медиана по типу занятости). Также из датасета были удалены дубликаты и аномальные значение. Для категоризации данных были добавлены категории людей по доходу (5 категорий) и категории цели кредита (4 категории). 

- Далее мы приступили к проверке первой гипотезы - поиск закономерности между возвратом кредита в срок и колличеством детей. Анализ показал, что сравнивая самые большие по численности категории семей, процент возврата вредитов рос вместе с колличеством детей. Так, самый низкий результат был у семей без детей, порядка 75%. Категории семей с одним и двумя детьми показали результат выше - 92% и 94% соответсвенно. 

- Приступаем к проверке следующей гиппотезы - зависимость между семейным положением и возврата кредита в срок. Результат показал, что люди, не находящиеся в отношениях, имеют большей процент возврата кредита в срок - 97%. Самые низкие результаты показали люди разведенные - 70%, овдавевшие - 66% или женатые - 75%. Также высокий показатель возврата средств у людей в гражданском браке - 93%. Исходя из исследования, можно сделать вывод, что наибольший показатель возврата денежных средств вовремя, демонстрируют люди без отношений или в гражданском браке. 

- В следующем исследовании мы анализируем зависимость между уровнем дохода и возвратом кредита в срок. Разделив доходы на категории, в выборке образовались две самые многочисленные группы людей - группа В имеет доход от 200 до 1 млн. рублей, и группа С с доходом от 50 до 200 тыс. рублей. Согласно данным, граждане, имеющие более низкие доходы, чаще стараются вернуть вовремя кредитные средства - 84%, для людей с уровнем дохода от 200 до 1 млн. рублей процент составил только 70%. 

- В следующем исследовании мы ищем зависимость между целью кредита и возвратом денег в срок. Согласно данным, самыми рискованными кредитами являются сделки с недвижимостью - 72% воозврата денежных средств в срок, и кредиты на проведение свадеб - 79%. Кредиты на образование и покупку машины, являются самыми безопастными - 79% для образования и 93% для автомобилей. 

- Можно сделать общий вывод, что приоритетными для банка кредиты, должны выдаваться людям с детьми, не состоящих в браке, с уровнем дохода до 200 тыс. рублей. Приоритетные цели кредита - образование и покупка автомобиля. Каких кредитов стоит опаться - в число рискованныых категорий попадают люди без детей, разведенных или овдавевшие, с доходом более 200 тыс. рублей. Самыми рискованными категориями являются сделки с недвижимостью и кредиты на свадьбу. Для более точной обработки данных, могу порекомендовать в столбце с доходами и днями стажа, ввести специальное обозначение для неработающих людей.